### MLFlow - Example ElasticNet Tracking

In [ ]:
## Imports

In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


In [15]:
def config():
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
def download_data():
     # Read the wine-quality csv file from the URL
    csv_url =\
        'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    try:
        data = pd.read_csv(csv_url, sep=';')
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e)
    
    return data

def prep_data(data):
    
    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha =  0.1
    l1_ratio =  0.1
    
    return train_x, train_y, test_x, test_y, alpha, l1_ratio

In [16]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

def train_model(train_x, train_y, test_x, test_y, alpha, l1_ratio):

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
        else:
            mlflow.sklearn.log_model(lr, "model")
            
            


In [18]:
def grid_search(train_x, train_y, test_x, test_y):
    
    min_l1 = 1
    max_l1 = 9
    min_alpha = 1
    max_alpha = 9
    
    for l1 in range(min_l1,max_alpha):
        
        
        for alpha in range(min_l1,max_l1):
                print('Training for Alpha {} and L1 {}'.format(float(alpha/10), float(l1/10)))
                train_model(train_x, train_y, test_x, test_y, float(alpha/10), float(l1/10))
            

In [19]:
if __name__ == "__main__":
    config()
    data = download_data()
    train_x, train_y, test_x, test_y, alpha, l1_ratio = prep_data(data)
    grid_search(train_x, train_y, test_x, test_y)
#     train_model(train_x, train_y, test_x, test_y, alpha, l1_ratio)

Training for Alpha 0.1 and L1 0.1
Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7128829045893679
  MAE: 0.5462202174984665
  R2: 0.2799376066653344
Training for Alpha 0.2 and L1 0.1
Elasticnet model (alpha=0.200000, l1_ratio=0.100000):
  RMSE: 0.7268133518615141
  MAE: 0.5586842416161893
  R2: 0.25152116688155723
Training for Alpha 0.3 and L1 0.1
Elasticnet model (alpha=0.300000, l1_ratio=0.100000):
  RMSE: 0.7347397539240514
  MAE: 0.5657315547549873
  R2: 0.23510678899596116
Training for Alpha 0.4 and L1 0.1
Elasticnet model (alpha=0.400000, l1_ratio=0.100000):
  RMSE: 0.7410782793160982
  MAE: 0.5712718681984226
  R2: 0.22185255063708886
Training for Alpha 0.5 and L1 0.1
Elasticnet model (alpha=0.500000, l1_ratio=0.100000):
  RMSE: 0.7460550348172179
  MAE: 0.576381895873763
  R2: 0.21136606570632266
Training for Alpha 0.6 and L1 0.1
Elasticnet model (alpha=0.600000, l1_ratio=0.100000):
  RMSE: 0.7510866447955419
  MAE: 0.5815681289333974
  R2: 0.20069264568704703


Elasticnet model (alpha=0.200000, l1_ratio=0.700000):
  RMSE: 0.7499835110445395
  MAE: 0.5819389930665501
  R2: 0.20303883413454016
Training for Alpha 0.3 and L1 0.7
Elasticnet model (alpha=0.300000, l1_ratio=0.700000):
  RMSE: 0.774713648356711
  MAE: 0.607967853255621
  R2: 0.14961391810397706
Training for Alpha 0.4 and L1 0.7
Elasticnet model (alpha=0.400000, l1_ratio=0.700000):
  RMSE: 0.8004478857657858
  MAE: 0.6350378679245181
  R2: 0.09217977708630032
Training for Alpha 0.5 and L1 0.7
Elasticnet model (alpha=0.500000, l1_ratio=0.700000):
  RMSE: 0.829586285479097
  MAE: 0.6627028304266674
  R2: 0.024882710417618137
Training for Alpha 0.6 and L1 0.7
Elasticnet model (alpha=0.600000, l1_ratio=0.700000):
  RMSE: 0.8321502650365332
  MAE: 0.6662000872414003
  R2: 0.018845859373919027
Training for Alpha 0.7 and L1 0.7
Elasticnet model (alpha=0.700000, l1_ratio=0.700000):
  RMSE: 0.832725785743381
  MAE: 0.667898097502809
  R2: 0.017488244494447747
Training for Alpha 0.8 and L1 0.7


In [11]:
mlflow.get_tracking_uri()

'file:///Users/ramine.tinati/git/MLFlow-Example/mlruns'